In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import random
import os

In [10]:
SEED = 42 
# Python RNG 
random.seed(SEED) 
# NumPy RNG 
np.random.seed(SEED) 
# Optional: full determinism for sklearn parallel algorithms 
os.environ["PYTHONHASHSEED"] = str(SEED) 
os.environ["OMP_NUM_THREADS"] = "1" 
os.environ["MKL_NUM_THREADS"] = "1"

In [ ]:
df = pd.read_csv('../data/data_cleaned.csv')


In [12]:
#usuwanie i wypełnianie pustych danych 

missing_data = (df.isnull().sum()/len(df)) *100
missing_data = missing_data[missing_data>0].sort_values(ascending = False).reset_index()
missing_data.columns = ["Cecha","Procent"]


# do modyfikacji w zależności od ostatecznej potrzeby (jeszcze idk jakie będzią ostatecznie)
# w %
col_cutoff_treshold = 48
col_trim_treshhold = 2


cols_to_drop  = missing_data[missing_data['Procent'] > col_cutoff_treshold]['Cecha'].tolist()
cols_to_trim = missing_data[missing_data['Procent'] < col_trim_treshhold]['Cecha'].tolist()

df.drop(columns=cols_to_drop,inplace=True)
df.dropna(subset=cols_to_trim,inplace=True)

In [13]:
unique_treshold = 0.90
# te zmienne wprowadzały szum wcześniej ale jak dojdę do ogarniania feature engineering to coś z nimi zrobię obiecuje
cols_to_drop  = [col for col in df.select_dtypes(include='object').columns if df[col].nunique()/len(df) > unique_treshold or df[col].nunique() == 1 ]
df.drop(columns=['model','model_version','body_color_original'], inplace=True)

df.drop(columns=cols_to_drop,inplace=True)


In [14]:
target_col = 'price'

# podział kolumn ze względu na typ
numeric_cols = df.select_dtypes(include=["float","int"]).columns.tolist()

bool_cols = df.select_dtypes(include=["bool"]).columns.tolist()
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()


#dopiero w tym miejscu żeby na wykresie było widać
numeric_cols.remove(target_col)

In [15]:
def fill_cols(data,numeric_cols,categorical_cols,stats):
    for col in numeric_cols:
        if col == 'gears' or col == 'nr_prev_owners':
            data[col] = data[col].fillna(0)
        else :
            data[col] = data[col].fillna(stats[col])
    for col,_ in categorical_cols:
        data[col] = data[col].fillna(stats[col])
    return data


def preprocess(data,numeric_cols,categorical_cols,scaler,stats):
    data = fill_cols(data,numeric_cols,categorical_cols,stats)
    data[numeric_cols] = scaler.transform(data[numeric_cols])
    for col,vals in categorical_cols:
        data[col] = pd.Categorical(data[col], categories=vals)
    data = pd.get_dummies(data, drop_first=True)
    return data


In [16]:
class Seeded:
    def __init__(self, estimator):
        self.estimator = estimator

    def __call__(self, *args, **kwargs):
        # Case 1: estimator is a class (e.g., RandomForestRegressor)
        if hasattr(self.estimator, "get_params"):
            # Instantiate a temporary object to inspect parameters
            params = self.estimator().get_params()
            if "random_state" in params and "random_state" not in kwargs:
                kwargs["random_state"] = SEED
            return self.estimator(*args, **kwargs)

        # Case 2: estimator is a function (e.g., train_test_split)
        if hasattr(self.estimator, "__code__"):
            if "random_state" in self.estimator.__code__.co_varnames and "random_state" not in kwargs:
                kwargs["random_state"] = SEED
            return self.estimator(*args, **kwargs)

        # Fallback
        return self.estimator(*args, **kwargs)

In [17]:
X, y = df.drop(target_col, axis=1), df[target_col]

TTS = Seeded(train_test_split)
X_train, X_test, y_train, y_test = TTS(X, y, test_size=0.2)
X_subtest, X_valid, y_subtest, y_valid = TTS(X_test, y_test, test_size=0.5)

scaler = StandardScaler()
scaler.fit(X_train[numeric_cols])

categorical_cols_with_vals = [ (col,[val for val in X_train[col].unique().tolist() if pd.notna(val)] ) for col in categorical_cols]

# to robię by uniknąć data leakage (pewnie poprawię potem by ładniej wyglądało)
stats = {col: X_train[col].median() for col in numeric_cols}
stats.update({col: X_train[col].mode()[0] for col in categorical_cols})

# tutaj gotowe dane do karmienia modelu
X_train_preprocessed = preprocess(X_train,numeric_cols,categorical_cols_with_vals,scaler,stats)
X_test_preprocessed = preprocess(X_test,numeric_cols,categorical_cols_with_vals,scaler,stats)
X_subtest_preprocessed = preprocess(X_subtest,numeric_cols,categorical_cols_with_vals,scaler,stats)

In [18]:
baseline = np.ones(len(y))*np.average(y)
mse_baseline = metrics.mean_squared_error(y, baseline)

clf = linear_model.LinearRegression()
clf.fit(X_train_preprocessed, y_train)

y_pred = clf.predict(X_test_preprocessed)
mse_st = metrics.mean_squared_error(y_test, y_pred)

print(mse_baseline / mse_st)

3.467155259234633


In [19]:
models = {
    "Linear (double split)": linear_model.LinearRegression(),
    "Ridge": Seeded(linear_model.Ridge)(alpha=0.1),
    "Lasso": Seeded(linear_model.Lasso)(alpha=0.01),
    "kNN": KNeighborsRegressor(n_neighbors=50, weights="distance"),
    "ElasticNet": Seeded(linear_model.ElasticNet)(alpha=0.1, l1_ratio=0.5),
    "RandomForestRegressor": Seeded(RandomForestRegressor)(n_estimators=100, max_depth=20)
}

results = {}
for name, model in models.items():
    model.fit(X_train_preprocessed, y_train)
    y_val_pred = model.predict(X_subtest_preprocessed)
    mse_val_best = metrics.mean_squared_error(y_subtest, y_val_pred)
    results[name] = mse_val_best
    print(f"{name}: MSE walidacja = {mse_val_best:.4f}")

best_model_name = min(results, key=results.get)
print(f"Najlepszy model: {best_model_name}")

y_pred = models[best_model_name].predict(X_test_preprocessed)
mse_val_best = metrics.mean_squared_error(y_test, y_pred)
print(mse_st / mse_val_best)

rmse = np.sqrt(mse_val_best)
mae  = metrics.mean_absolute_error(y_test, y_pred)
r2   = metrics.r2_score(y_test, y_pred)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred)

print("\n=== Ewaluacja najlepszego modelu (test) ===")
print(f"MSE:   {mse_val_best:.2f}")
print(f"RMSE:  {rmse:.2f}")
print(f"MAE:   {mae:.2f}")
print(f"R2:    {r2:.4f}")
print(f"MAPE:  {mape*100:.2f}%")

Linear (double split): MSE walidacja = 4274689367.6857
Ridge: MSE walidacja = 4267546199.3754
Lasso: MSE walidacja = 4274677094.3301
kNN: MSE walidacja = 2893316010.1203
ElasticNet: MSE walidacja = 5170330334.9276
RandomForestRegressor: MSE walidacja = 2646255457.4766
Najlepszy model: RandomForestRegressor
1.7520247859594231

=== Ewaluacja najlepszego modelu (test) ===
MSE:   1859940305.68
RMSE:  43127.03
MAE:   7460.47
R2:    0.7333
MAPE:  83.80%


In [20]:
### Log-ification of database

results_log = {}
for name, model in models.items():
    model.fit(X_train_preprocessed, np.log1p(y_train))
    y_val_pred_log = model.predict(X_subtest_preprocessed)
    y_val_pred = np.expm1(y_val_pred_log)
    mse_val = metrics.mean_squared_error(y_subtest, y_val_pred)
    results_log[name] = mse_val
    print(f"{name}: MSE walidacja = {mse_val:.4f}")

best_model = models[best_model_name]
print(f"Najlepszy model (skala log.): {best_model_name}")
best_model.fit(X_train_preprocessed, np.log1p(y_train))

y_pred_log = best_model.predict(X_test_preprocessed)
y_pred = np.expm1(y_pred_log)

mse_log = metrics.mean_squared_error(y_test, y_pred)
print("MSE test:", mse_val_best / mse_log)

rmse = np.sqrt(mse_log)
mae  = metrics.mean_absolute_error(y_test, y_pred)
r2   = metrics.r2_score(y_test, y_pred)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred)

print("\n=== Ewaluacja najlepszego modelu (test) ===")
print(f"MSE:   {mse_log:.2f}")
print(f"RMSE:  {rmse:.2f}")
print(f"MAE:   {mae:.2f}")
print(f"R2:    {r2:.4f}")
print(f"MAPE:  {mape*100:.2f}%")

Linear (double split): MSE walidacja = 3499411842.5259
Ridge: MSE walidacja = 3495426445.6761
Lasso: MSE walidacja = 4213326995.7163
kNN: MSE walidacja = 3940587746.8179
ElasticNet: MSE walidacja = 4251126737.8177
RandomForestRegressor: MSE walidacja = 2522608193.2408
Najlepszy model (skala log.): RandomForestRegressor
MSE test: 1.0314195197198421

=== Ewaluacja najlepszego modelu (test) ===
MSE:   1803282049.76
RMSE:  42465.07
MAE:   7276.36
R2:    0.7414
MAPE:  77.08%


In [21]:
print (df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 113920 entries, 0 to 118381
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   price       113920 non-null  float64
 1   make        113920 non-null  object 
 2   car_age     113920 non-null  float64
 3   mileage_km  113920 non-null  float64
 4   power_kw    113920 non-null  float64
 5   gears       73105 non-null   float64
 6   cylinders   91936 non-null   float64
 7   is_new      113920 non-null  bool   
dtypes: bool(1), float64(6), object(1)
memory usage: 7.1+ MB
None
